In [14]:
import mlflow
import pandas as pd
import logging
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import time

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
352,I wasted 35 minutes of my life on this turkey ...,negative
644,I've watched a lot of television in my 51 year...,positive
916,This is probably one of those films too many c...,positive
200,"""Indian burial ground"": If those three words a...",negative
940,For me personally this film goes down in my to...,positive


In [3]:
# data preprocessing

def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:31: SyntaxWarning: invalid escape sequence '\s'
<>:31: SyntaxWarning: invalid escape sequence '\s'
C:\Users\rohit\AppData\Local\Temp\ipykernel_24680\3707620020.py:31: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
352,wasted minute life turkey gave up main charact...,negative
644,watched lot television year never much fun wee...,positive
916,probably one film many commented undiscovered ...,positive
200,indian burial ground three word appear anywher...,negative
940,personally film go top four time exception jam...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    266
positive    234
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
352,wasted minute life turkey gave up main charact...,0
644,watched lot television year never much fun wee...,1
916,probably one film many commented undiscovered ...,1
200,indian burial ground three word appear anywher...,0
940,personally film go top four time exception jam...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [29]:
vectorizer = CountVectorizer(max_features=200)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/rohitkr8527/sentiment-analysis.mlflow')
dagshub.init(repo_owner='rohitkr8527', repo_name='sentiment-analysis', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")


2025-08-29 13:30:53,081 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/rohitkr8527/sentiment-analysis "HTTP/1.1 200 OK"


Initialized MLflow to track repo "rohitkr8527/sentiment-analysis"

2025-08-29 13:30:53,086 - INFO - Initialized MLflow to track repo "rohitkr8527/sentiment-analysis"


Repository rohitkr8527/sentiment-analysis initialized!

2025-08-29 13:30:53,087 - INFO - Repository rohitkr8527/sentiment-analysis initialized!


<Experiment: artifact_location='mlflow-artifacts:/61da7ccc7aef45be883fbcabadf3f873', creation_time=1756453984341, experiment_id='0', last_update_time=1756453984341, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [31]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 200)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  
        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-29 13:36:34,655 - INFO - Starting MLflow run...
2025-08-29 13:36:35,088 - INFO - Logging preprocessing parameters...
2025-08-29 13:36:36,358 - INFO - Initializing Logistic Regression model...
2025-08-29 13:36:36,359 - INFO - Fitting the model...
2025-08-29 13:36:36,384 - INFO - Model training complete.
2025-08-29 13:36:36,385 - INFO - Logging model parameters...
2025-08-29 13:36:36,812 - INFO - Making predictions...
2025-08-29 13:36:36,813 - INFO - Calculating evaluation metrics...
2025-08-29 13:36:36,819 - INFO - Logging evaluation metrics...
2025-08-29 13:36:38,544 - INFO - Saving and logging the model...
2025/08/29 13:36:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-08-29 13:36:45,618 - INFO - Model training and logging completed in 10.53 seconds.
2025-08-29 13:36:45,619 - INFO - Accuracy: 0.73
2025-08-29 13:36:45,620 - INFO - Precision: 0

🏃 View run intelligent-croc-424 at: https://dagshub.com/rohitkr8527/sentiment-analysis.mlflow/#/experiments/0/runs/b96821cc44be4ea8ac32eb8d31e01913
🧪 View experiment at: https://dagshub.com/rohitkr8527/sentiment-analysis.mlflow/#/experiments/0
